# Exploratory Data Analysis

In [11]:
import os
for root, dirs, files in os.walk('../data/FLAME_Dataset_subset/'):
    print(files)

[]
[]
['resized_test_fire_frame1011.jpg', 'resized_test_fire_frame1031.jpg', 'resized_test_fire_frame1038.jpg', 'resized_test_fire_frame1040.jpg', 'resized_test_fire_frame1052.jpg', 'resized_test_fire_frame1057.jpg', 'resized_test_fire_frame1064.jpg', 'resized_test_fire_frame1075.jpg', 'resized_test_fire_frame1082.jpg', 'resized_test_fire_frame1083.jpg', 'resized_test_fire_frame1094.jpg', 'resized_test_fire_frame1099.jpg', 'resized_test_fire_frame1102.jpg', 'resized_test_fire_frame1120.jpg', 'resized_test_fire_frame113.jpg', 'resized_test_fire_frame1139.jpg', 'resized_test_fire_frame1153.jpg', 'resized_test_fire_frame1163.jpg', 'resized_test_fire_frame1183.jpg', 'resized_test_fire_frame120.jpg', 'resized_test_fire_frame1204.jpg', 'resized_test_fire_frame1216.jpg', 'resized_test_fire_frame1218.jpg', 'resized_test_fire_frame1225.jpg', 'resized_test_fire_frame1234.jpg', 'resized_test_fire_frame1235.jpg', 'resized_test_fire_frame1256.jpg', 'resized_test_fire_frame1267.jpg', 'resized_test_f

In [19]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}-{}'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        filecount = len(files)
        if filecount>0:
            print(f'{subindent}-{filecount} files')

In [20]:
list_files('../data/FLAME_Dataset_subset')

-FLAME_Dataset_subset
    -Test
        -Fire
            -353 files
        -No_Fire
            -211 files
    -Training
        -Fire
            -1574 files
        -No_Fire
            -889 files


In [24]:
list_files('../data/FLAME Dataset - Shamsoshoara/')

-
-Test
    -Fire
        -5137 files
    -No_Fire
        -3480 files
-Training
    -Fire
        -25018 files
    -No_Fire
        -14357 files
